In [9]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
import warnings


from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore",category=DeprecationWarning)

%matplotlib inline

In [55]:
dataset = pd.read_csv('https://raw.githubusercontent.com/akshatha-swamy/sentiment-analysis/main/dataset.csv?token=GHSAT0AAAAAACGPRHC23UDCUQXGMF2PLPU4ZIK7GVA')

In [56]:
dataset

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [57]:
negative_tweets = dataset[dataset['label'] == 1]

# Display a sample of negative tweets (e.g., the first 5 rows)
print(negative_tweets['tweet'].head())

13    @user #cnn calls #michigan middle school 'buil...
14    no comment!  in #australia   #opkillingbay #se...
17                               retweet if you agree! 
23      @user @user lumpy says i am a . prove it lumpy.
34    it's unbelievable that in the 21st century we'...
Name: tweet, dtype: object


In [58]:
def remove_pattern(text,pattern):

    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)

    # re.sub() removes @user from the sentences in the dataset
    for i in r:
        text = re.sub(i,"",text)

    return text

In [59]:
dataset['Tidy_Tweets'] = np.vectorize(remove_pattern)(dataset['tweet'], "@[\w]*")
dataset['Tidy_Tweets'] = dataset['Tidy_Tweets'].str.replace("[^a-zA-Z#]", " ", regex=True)
dataset['Tidy_Tweets'] = dataset['Tidy_Tweets'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
tokenized_tweet = dataset['Tidy_Tweets'].apply(lambda x: x.split())
tokenized_tweet.head()


0    [when, father, dysfunctional, selfish, drags, ...
1    [thanks, #lyft, credit, cause, they, offer, wh...
2                              [bihday, your, majesty]
3                     [#model, love, take, with, time]
4                   [factsguide, society, #motivation]
Name: Tidy_Tweets, dtype: object

In [60]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

dataset['Tidy_Tweets'] = tokenized_tweet
dataset.head()

,id,label,tweet,Tidy_Tweets
0,1,0,@user when a father is dysfunctional and is s...,when father dysfunctional selfish drags kids i...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks #lyft credit cause they offer wheelchai...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,#model love take with time
4,5,0,factsguide: society now #motivation,factsguide society #motivation


In [61]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(dataset['Tidy_Tweets'])

df_bow = pd.DataFrame(bow.todense())
df_bow


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31958,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
train_bow = bow[:31962]

train_bow.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [63]:
x_train_bow, x_valid_bow, y_train_bow, y_valid_bow = train_test_split(train_bow,dataset['label'],test_size=0.3,random_state=2)

In [64]:
from sklearn.metrics import f1_score

In [65]:
from sklearn.linear_model import LogisticRegression
Log_Reg = LogisticRegression(random_state=0,solver='lbfgs')

In [66]:
Log_Reg.fit(x_train_bow,y_train_bow)

LogisticRegression(random_state=0)

In [67]:
prediction_bow = Log_Reg.predict_proba(x_valid_bow)

prediction_bow

array([[9.49524422e-01, 5.04755785e-02],
       [9.99318114e-01, 6.81886135e-04],
       [9.11845968e-01, 8.81540316e-02],
       ...,
       [8.53451535e-01, 1.46548465e-01],
       [9.59720119e-01, 4.02798806e-02],
       [9.68807903e-01, 3.11920969e-02]])

In [68]:
# if prediction is greater than or equal to 0.3 than 1 else 0
# Where 0 is for positive sentiment tweets and 1 for negative sentiment tweets
prediction_int = prediction_bow[:,1]>=0.3

# converting the results to integer type
prediction_int = prediction_int.astype(np.int)
prediction_int

# calculating f1 score
log_bow = f1_score(y_valid_bow, prediction_int)

log_bow

0.528174936921783

In [69]:
from sklearn.metrics import accuracy_score

# Assuming 'y_true' is a list or array of true labels and 'y_pred' is a list or array of predicted labels
accuracy = accuracy_score(y_valid_bow,prediction_int)

In [70]:
accuracy

0.9414954635519867

In [71]:
# Input statement
input_statement = "racist"

# Preprocess the input statement
input_statement = remove_pattern(input_statement, "@[\w]*")  # Remove user mentions
input_statement = re.sub("[^a-zA-Z#]", " ", input_statement)  # Remove special characters
input_statement = ' '.join([w for w in input_statement.split() if len(w) > 3])  # Remove short words

# Tokenize the input statement
tokenized_input = input_statement.split()
input_statement = ' '.join(tokenized_input)

# Vectorize the input statement using the same vectorizer used for training data
input_vector = bow_vectorizer.transform([input_statement])

# Predict the label for the input statement
predicted_label = Log_Reg.predict(input_vector)

if predicted_label[0] == 0:
    print("positive Sentiment")
else:
    print("negative Sentiment")


negative Sentiment
